In [1]:
import sys
import os
PATH_TO_PROJECT = "/home/tanawatp/volta_ws/src/volta_controller"
sys.path.append(PATH_TO_PROJECT)
from Utils.UtilsFunc import*
from Utils.KinematicsFunc import*
from pydrake.common.eigen_geometry import Quaternion
from pydrake.math import RigidTransform, RotationMatrix, RollPitchYaw

In [2]:
p,i = DefinePlant(model_name="Volta_rev4",
                  is_fixed=True,
                  fixed_frame="torso")
desire_torso_height = 0.46
q_L = SolveInverseKinematics(model_series="Volta",
                             plant=p,
                             is_left=True,
                             base_P_e=np.array([0.0, 0.125, -desire_torso_height]),
                             base_R_e=RotationMatrix.MakeXRotation(0.0))
q_R = SolveInverseKinematics(model_series="Volta",
                             plant=p,
                             is_left=False,
                             base_P_e=np.array([0.0, -0.125, -desire_torso_height]),
                             base_R_e=RotationMatrix.MakeXRotation(0.0))

In [3]:
q_L

(True,
 array([-0.55373151,  0.        ,  0.        , -1.43701774, -0.88297172,
         0.        ]))

In [4]:
q_R

(True,
 array([0.5536631 , 0.        , 0.        , 1.43695963, 0.88306167,
        0.        ]))

In [5]:
p.CalcTotalMass(p.CreateDefaultContext())

5.001999999999999

In [ ]:

R = RotationMatrix.MakeZRotation(0.0)
print(R)
Q = Quaternion(R.matrix())
print(Q.wxyz())

RotationMatrix([
  [1.0, -0.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
])
[1. 0. 0. 0.]


In [ ]:
T_B = RigidTransform(RotationMatrix.MakeXRotation(np.pi/2.0), [0, 0, 0.0])
print(T_B)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 6.123233995736766e-17, -1.0],
    [0.0, 1.0, 6.123233995736766e-17],
  ]),
  p=[0.0, 0.0, 0.0],
)


In [ ]:

T_C = RigidTransform(RotationMatrix.MakeZRotation(2.62+np.pi/2.0), [0, 0, -0.5])
print(T_C)

RigidTransform(
  R=RotationMatrix([
    [-0.4982616424118386, 0.8670267214458024, 0.0],
    [-0.8670267214458024, -0.4982616424118386, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, -0.5],
)


In [9]:
np.sin(2.62+np.pi/2.0)

np.float64(-0.8670267214458024)

In [10]:
np.cos(2.62+np.pi/2.0)

np.float64(-0.4982616424118386)

In [ ]:
T = T_B.multiply(T_C)
print(T)

RigidTransform(
  R=RotationMatrix([
    [-0.4982616424118386, 0.8670267214458024, 0.0],
    [-5.3090074959691287e-17, -3.0509726275878066e-17, -1.0],
    [-0.8670267214458024, -0.4982616424118386, 6.123233995736766e-17],
  ]),
  p=[0.0, 0.5, -3.061616997868383e-17],
)


In [11]:
T_D = RigidTransform(RotationMatrix.MakeXRotation(np.pi/2.0), [1.0, 0.0, 0.0])
T_E = RigidTransform(RotationMatrix.MakeZRotation(0), [0, 0, 1.0])
print(T_D.multiply(T_E))

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 6.123233995736766e-17, -1.0],
    [0.0, 1.0, 6.123233995736766e-17],
  ]),
  p=[1.0, -1.0, 6.123233995736766e-17],
)


In [12]:
T_D

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 6.123233995736766e-17, -1.0],
    [0.0, 1.0, 6.123233995736766e-17],
  ]),
  p=[1.0, 0.0, 0.0],
)

In [1]:
from statemachine import StateMachine, State
import numpy as np

class RobotControllerStateMachine(StateMachine):
    """
    State machine for robot controller with different operational modes
    """
    
    # Define states
    disable_controller = State('DisableController', initial=True)
    hold_position = State('HoldPosition')
    home = State('Home')
    walking = State('walkExecution')
    
    # Define transitions based on operation_command values
    # operation_command: -1=disable, 0=home, 1=walking, 2=hold_position
    
    # From disable_controller
    to_home_from_disable = disable_controller.to(home)
    to_walk_from_disable = disable_controller.to(walking)
    to_hold_from_disable = disable_controller.to(hold_position)
    stay_disabled = disable_controller.to(disable_controller)
    
    # From hold_position
    to_disable_from_hold = hold_position.to(disable_controller)
    to_home_from_hold = hold_position.to(home)
    to_walk_from_hold = hold_position.to(walking)
    stay_holding = hold_position.to(hold_position)
    
    # From home
    to_disable_from_home = home.to(disable_controller)
    to_walk_from_home = home.to(walking)
    to_hold_from_home = home.to(hold_position)
    stay_home = home.to(home)
    
    # From walking
    to_disable_from_walk = walking.to(disable_controller)
    to_home_from_walk = walking.to(home)
    to_hold_from_walk = walking.to(hold_position)
    stay_walk = walking.to(walking)
    
    def __init__(self):
        super().__init__()
        
        # Robot state variables
        self.controller_enable = False
        self.operation_command = -1
        self.current_position = np.zeros(12)  # 6 DOF per leg
        
        # Joint states
        self.L_leg_q = np.zeros(6)
        self.R_leg_q = np.zeros(6)
        self.L_leg_qd = np.zeros(6)
        self.R_leg_qd = np.zeros(6)
        
        # Home state management
        self.home_state = "init_trajectories"
        self.ind = 0
        self.trajectory_time = 0.0
        self.home_interval = 2.0  # Example value
        self.timer_period = 0.01  # Example value
        
        # Trajectory objects (placeholder - implement based on your InitCubicTrajectories)
        self.traj_L = None
        self.traj_R = None
        
        # Initial positions (placeholder - set these based on your robot)
        self.q_init_L = np.zeros((6, 1))
        self.q_init_R = np.zeros((6, 1))
        self.qd_init_L = np.zeros((6, 1))
        self.qd_init_R = np.zeros((6, 1))
        self.temp = 0.0
    def set_operation_command(self, command):
        """Set walk command and trigger appropriate transition"""
        self.operation_command = command
        self._process_operation_command()
    
    def _process_operation_command(self):
        """Process walk command and trigger appropriate state transition"""
        if self.operation_command == -1:
            self._transition_to_disable()
        elif self.operation_command == 0:
            self._transition_to_home()
        elif self.operation_command == 1:
            self._transition_to_walk()
        elif self.operation_command == 2:
            self._transition_to_hold()
    
    def _transition_to_disable(self):
        """Transition to disable controller state"""
        if self.current_state == self.hold_position:
            self.to_disable_from_hold()
        elif self.current_state == self.home:
            self.to_disable_from_home()
        elif self.current_state == self.walking:
            self.to_disable_from_walk()
        else:
            self.stay_disabled()
    
    def _transition_to_home(self):
        """Transition to home state"""
        if self.current_state == self.disable_controller:
            self.to_home_from_disable()
        elif self.current_state == self.hold_position:
            self.to_home_from_hold()
        elif self.current_state == self.walking:
            self.to_home_from_walk()
        else:
            self.stay_home()
    
    def _transition_to_walk(self):
        """Transition to walk execution state"""
        if self.current_state == self.disable_controller:
            self.to_walk_from_disable()
        elif self.current_state == self.hold_position:
            self.to_walk_from_hold()
        elif self.current_state == self.home:
            self.to_walk_from_home()
        else:
            self.stay_walk()
    
    def _transition_to_hold(self):
        """Transition to hold position state"""
        if self.current_state == self.disable_controller:
            self.to_hold_from_disable()
        elif self.current_state == self.home:
            self.to_hold_from_home()
        elif self.current_state == self.walking:
            self.to_hold_from_walk()
        else:
            self.stay_holding()
    
    # State entry actions using decorator syntax
    @disable_controller.enter
    def enter_disable_controller(self):
        """Actions when entering disable_controller state"""
        print("Entering disable_controller state")
        self.controller_enable = False
    
    @hold_position.enter
    def enter_hold_position(self):
        """Actions when entering hold_position state"""
        print("Entering hold_position state")
        self.controller_enable = True
        self.L_leg_q = np.array(self.current_position)[0:6]
        self.R_leg_q = np.array(self.current_position)[6:12]
        self.L_leg_qd = np.zeros(6)
        self.R_leg_qd = np.zeros(6)
    
    @home.enter
    def enter_home(self):
        """Actions when entering home state"""
        print("Entering home state")
        self.temp += 1.0
        print(f"Home state entered {self.temp} times")
        self.home_state = "init_trajectories"
        self._init_home_trajectories()
    
    @walking.enter
    def enter_walking(self):
        """Actions when entering walking state"""
        print(self.temp)
        print("Entering walking state")
        self.controller_enable = True
    
    # Home state sub-state management
    def _init_home_trajectories(self):
        """Initialize trajectories for homing"""
        self.controller_enable = False
        self.ind = 0
        self.trajectory_time = 0.0
        
        # Get target positions
        q_L_f = self.q_init_L[:, self.ind]
        q_R_f = self.q_init_R[:, self.ind]
        qd_L_f = self.qd_init_L[:, self.ind]
        qd_R_f = self.qd_init_R[:, self.ind]
        
        q_0 = np.array(self.current_position)
        
        # Initialize trajectories (you'll need to implement InitCubicTrajectories)
        # This is a placeholder for your trajectory initialization
        P_L = np.array([q_0[0:6], q_L_f]).T
        V_L = np.array([np.zeros(6), np.zeros(6)]).T
        
        P_R = np.array([q_0[6:12], q_R_f]).T
        V_R = np.array([np.zeros(6), np.zeros(6)]).T
        
        # self.traj_L = InitCubicTrajectories(P=P_L, V=V_L, 
        #                                    Times=[0, self.home_interval], n=6)
        # self.traj_R = InitCubicTrajectories(P=P_R, V=V_R,
        #                                    Times=[0, self.home_interval], n=6)
        
        self.home_state = "evaluate_trajectories"
    
    def update_home_state(self):
        """Update home state - call this in your main loop when in home state"""
        if self.current_state != self.home:
            return
            
        if self.home_state == "evaluate_trajectories":
            self.controller_enable = True
            
            if self.trajectory_time <= self.home_interval:
                # Evaluate trajectories (placeholder - implement based on your trajectory class)
                for i in range(6):  # Assuming 6 DOF per leg
                    # self.L_leg_q[i] = self.traj_L[i].value(self.trajectory_time)[0][0]
                    # self.R_leg_q[i] = self.traj_R[i].value(self.trajectory_time)[0][0]
                    # self.L_leg_qd[i] = self.traj_L[i].EvalDerivative(self.trajectory_time, 1)[0][0]
                    # self.R_leg_qd[i] = self.traj_R[i].EvalDerivative(self.trajectory_time, 1)[0][0]
                    pass
                
                self.trajectory_time += self.timer_period
            else:
                self.home_state = "done"
                self.set_operation_command(2)  # Transition to hold_position
    
    def update_walking(self):
        """Update walk execution - call this in your main loop when in walk execution state"""
        if self.current_state == self.walking:
            self.UpdateDesiredJointStates()
    
    def UpdateDesiredJointStates(self):
        """Update desired joint states - implement your walk execution logic here"""
        # Placeholder for your walk execution logic
        pass
    
    def get_current_state_name(self):
        """Get the current state name as string"""
        return self.current_state.name
    
    def is_controller_enabled(self):
        """Check if controller is enabled"""
        return self.controller_enable


# Example usage
if __name__ == "__main__":
    # Create robot controller state machine
    robot = RobotControllerStateMachine()
    
    print(f"Initial state: {robot.get_current_state_name()}")
    print(f"Controller enabled: {robot.is_controller_enabled()}")
    
    # Test state transitions
    print("\n--- Testing state transitions ---")
    
    # Go to home
    robot.set_operation_command(0)
    print(f"After home command: {robot.get_current_state_name()}")
    
    # Go to walk execution
    robot.set_operation_command(1)
    print(f"After walk command: {robot.get_current_state_name()}")
    
    # Go to hold position
    robot.set_operation_command(2)
    print(f"After hold command: {robot.get_current_state_name()}")
    
    # Disable controller
    robot.set_operation_command(-1)
    print(f"After disable command: {robot.get_current_state_name()}")
    
    print(f"Controller enabled: {robot.is_controller_enabled()}")

Entering disable_controller state
Initial state: DisableController
Controller enabled: False

--- Testing state transitions ---
Entering home state
Home state entered 1.0 times
After home command: Home
1.0
Entering walking state
After walk command: walkExecution
Entering hold_position state
After hold command: HoldPosition
Entering disable_controller state
After disable command: DisableController
Controller enabled: False


In [3]:
while True:
    user_input = int(input("Enter operation command (-1=disable, 0=home, 1=walking, 2=hold_position, q=quit): "))
    # print(type(user_input))
    # print(user_input)
    print(robot.get_current_state_name)
    if user_input == 99:
        break
    robot.set_operation_command(user_input)

<bound method RobotControllerStateMachine.get_current_state_name of RobotControllerStateMachine(model=Model(state=disable_controller), state_field='state', current_state='disable_controller')>


In [3]:
s = "0"
print(int(s))

0
